In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd /home/naodell/work/hgcal/analysis

/home/naodell/work/hgcal/analysis


# Event Mixer prototyping

In this notebook, I will test out the event mixing machinery.  The idea here is to save time and CPU cycles by taking simulated events that have been unpacked and converted to dataframes and then mixing events that are in a single common hexagonal neighborhood.  The neighborhood is defined by the central hexagonal coordinate $(u,v)$ and the six neighboring hexagons.

In [2]:
import os
import pickle
from glob import glob

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import utils.geometry_tools as gt
import utils.plot_tools as pt
#from scipy.optimize import lsq_linear
#from sklearn.linear_model import LinearRegression

matplotlib.rcParams.update({'font.size': 18, 'figure.facecolor':'white', 'figure.figsize':(8, 8)})


First, select all events that lie in the seven hexagon neighborhood centered on $(u, v)$ and save them to a single dataframe for sampling.

In [3]:
uv = (5, 2)
hex_neighborhood = gt.hex_neighbors(uv)
cuts = 'tc_zside == -1 and tc_subdet == 1  and tc_energy > 0.1'
list_df_tc = []
list_df_gen = []
for filename in tqdm(glob('local_data/tc_data/*')):
    # get the data
    input_file = open(filename, 'rb')
    data_dict = pickle.load(input_file)
    df_tc = data_dict['tc']
    df_gen = data_dict['gen'].query('genpart_exeta < 0.')

    # apply some cuts
    df_cut = df_tc.query(cuts)

    # get events in neighborhood
    events = gt.get_events_in_neighborhood(uv, df_cut)
    
    list_df_tc.append(df_cut.loc[events])
    #list_df_gen.append(df_gen.loc[events])
    
df_events = pd.concat(list_df_tc)

  0%|          | 0/52 [00:00<?, ?it/s]

Next, event mixtures are generated by subsampling the event dataframe.  First, a Poisson variate is drawn to determine the number of events (photons in this case) to include and those events are selected from the full event dataframe.  Then the events are combined by grouping them based on $(layer, u_{wafer}, v_{wafer}, u_{cell}, v_{cell})$ and summing their contents (this probably needs some refinement).  Finally, the events are saved as both pandas series and numpy arrays.  The numpy arrays are populated according to the hex to square grid conversion from `geometry_tools`. 

In [4]:
### mix events ###

n_events = 10000
event_ix = df_events.index.unique(level=0)
event_mixtures_series = []
event_mixtures_array = []
neighbor_mixtures = []
wafer_uv = ['tc_layer', 'tc_waferu', 'tc_waferv', 'tc_cellu', 'tc_cellv']
for i in tqdm(range(n_events)):
    
    # get mixtures
    size = np.max([np.random.poisson(10), 1])
    events = np.random.choice(event_ix, size=size, replace=False)
    df_mix = df_events.loc[events]
    s_energy = df_mix.groupby(wafer_uv).sum()['tc_energy']
    event_mixtures_series.append(s_energy)
    
    # get wafer with highest energy across all layers
    wafer_sums = df_mix.groupby(wafer_uv[:3]).sum()['tc_energy']
    uv_max = wafer_sums.idxmax()
    array_mix = gt.convert_wafer_to_array(s_energy.loc[:, uv_max[1], uv_max[2]])
    event_mixtures_array.append([array_mix, (uv_max[1:])])
    
    # convert hex neighborhood to array
    neighbor_mix = gt.convert_wafer_neighborhood_to_array(s_energy, uv)
    neighbor_mixtures.append(neighbor_mix)
    
outfile = open('local_data/event_mixtures/single_photon_5_2_test.pkl', 'wb')
pickle.dump(neighbor_mixtures, outfile)                            
outfile.close()                                                 

  0%|          | 0/10000 [00:00<?, ?it/s]

Produce some visualizations to validate the mixing is working as expected.  _Expand this to juxtpose wafer based visualizations and locations of gen particles passing through the layer_

In [5]:
# Visualize the results
for i, neighbor_mix in enumerate(neighbor_mixtures[:20]):
    #print(i)
    fig = plt.figure(figsize=(20, 8))
    for j, layer_data in enumerate(neighbor_mix):
        ax = plt.subplot(2, 7, j+1)
        ax.imshow(layer_data, norm=matplotlib.colors.LogNorm(vmin=0.01))
        ax.set_title(f'layer {2*j + 1}')

    plt.tight_layout()
    #plt.show()
    plt.savefig(f'plots/event_mixes/mix_{i}.png')
    fig.clear()
    plt.close()
